# Transformer on MIMIC-IV (Demo)
This minimal walkthrough uses PyHealth's built-in utilities to train the PyHealth Transformer model on a MIMIC-IV mortality prediction task.

In [ ]:
from pyhealth.datasets import MIMIC4Dataset, get_dataloader, split_by_sample
from pyhealth.tasks import InHospitalMortalityMIMIC4
from pyhealth.models import Transformer
from pyhealth.trainer import Trainer

In [ ]:
dataset = MIMIC4Dataset(
    ehr_root="/home/logic/Github/mimic4",
    ehr_tables=["diagnoses_icd", "procedures_icd", "prescriptions", "labevents"],
    dev=True,
)

In [ ]:
task = InHospitalMortalityMIMIC4()
sample_dataset = dataset.set_task(
    task,
    cache_dir="../../test_cache_transformer_m4"
)
train_dataset, val_dataset, test_dataset = split_by_sample(sample_dataset, ratios=[0.7, 0.1, 0.2])

In [ ]:
train_loader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_loader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = Transformer(dataset=sample_dataset, embedding_dim=128, heads=2, num_layers=2, dropout=0.1)
trainer = Trainer(model=model, metrics=["roc_auc", "pr_auc"])

In [ ]:
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor="roc_auc",
    optimizer_params={"lr": 1e-4},
)

In [ ]:
trainer.evaluate(test_loader)